**Loading of libaries and dataset**

In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from datasets import load_dataset
import pandas as pd
import re

In [3]:
# Load train dataset
ds = load_dataset("ailsntua/QEvasion")

# Convert to pandas and keep only useful columns
df_train = ds["train"].to_pandas()[["question","interview_question",
                                    "interview_answer", "label","url"]]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/779 [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/14.9M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3448 [00:00<?, ? examples/s]

In [ ]:
df_train.head(5)

,question,interview_question,interview_answer,label
0,How would you respond to the accusation that t...,\nQ. Of the Biden administration. And accused ...,"\nThe President. Well, look, first of all, the...",Explicit
1,Do you think President Xi is being sincere abo...,\nQ. Of the Biden administration. And accused ...,"\nThe President. Well, look, first of all, the...",General
2,1. Q1: Do you believe the country's slowdown a...,\nQ. No worries. Do you believe the country's ...,"\nThe President. Look, I think China has a dif...",Partial/half-answer
3,2. Q2: Are you worried about the meeting betwe...,\nQ. No worries. Do you believe the country's ...,"\nThe President. Look, I think China has a dif...",Dodging
4,Is the President's engagement with Asian coun...,"\nQ. I can imagine. It is evening, I'd like to...","\nThe President. Well, I hope I get to see Mr....",Explicit


**Main preprocessing procedure**

In [4]:
def main_preprocessing(df: pd.DataFrame) -> pd.DataFrame:

    # Regex explanation:

    # ^ matches the start of the string
    # (\d+\.|Part \d+:|Q\d*:|\d+\. Q\d*: ) is a capturing group that matches
    # one of the following:
    #     \d+\. : one or more digits followed by a period
    #
    #     Part \d+: : the string "Part " followed by one or more digits,
    #     a colon, and an optional space
    #
    #     Q\d*: : the string "Q" followed by one or more digits, a colon, a
    #     and an optional space
    #
    #     \d+\. Q\d*: : one or more digits followed by a period, a space,
    #     "Q", one or more digits, a colon, and an optional space
    #
    #     - : start sentence with "-"

    # Remove indexing from questions
    index_pattern = r'^(\d+\. Q\d+:|\d+\.|Part \d+:|Q\d+:|-)'
    df['question'] = df['question'].str.replace(index_pattern,
                                                            '', regex=True)

    # Remove quotes and new line espace characters
    df['question'] = df['question'].str.replace(r'["\n]',
                                                            '', regex=True)
    df['interview_answer'] = df['interview_answer'].str.replace(
                                                                r'\n', '',
                                                                regex=True)

    # Remove first sentence from answer
    sentence_pattern = r'^[^.]+\.?'
    df['interview_answer'] = df['interview_answer'].str.replace(
                                                            sentence_pattern,
                                                            '', regex=True)

    # Remove description from questions

    # List of exception indexes
    exceptions = [142,493,699,809,1052,1053,1446,
                  2417,2631,2821,3181,3390]

    df.loc[~df.index.isin(exceptions), 'question'] = df_train.loc[
        ~df.index.isin(exceptions), 'question'].apply(
        lambda x: re.sub(r'^[^:]+: ', '', x))

    return df

In [5]:
df_train = main_preprocessing(df_train)

In [6]:
df_train.head(5)

,question,interview_question,interview_answer,label,url
0,How would you respond to the accusation that t...,\nQ. Of the Biden administration. And accused ...,"Well, look, first of all, the—I am sincere ab...",Explicit,https://www.presidency.ucsb.edu/documents/the-...
1,Do you think President Xi is being sincere abo...,\nQ. Of the Biden administration. And accused ...,"Well, look, first of all, the—I am sincere ab...",General,https://www.presidency.ucsb.edu/documents/the-...
2,Do you believe the country's slowdown and gro...,\nQ. No worries. Do you believe the country's ...,"Look, I think China has a difficult economic ...",Partial/half-answer,https://www.presidency.ucsb.edu/documents/the-...
3,Are you worried about the meeting between Pre...,\nQ. No worries. Do you believe the country's ...,"Look, I think China has a difficult economic ...",Dodging,https://www.presidency.ucsb.edu/documents/the-...
4,Is the President's engagement with Asian coun...,"\nQ. I can imagine. It is evening, I'd like to...","Well, I hope I get to see Mr. Xi sooner than ...",Explicit,https://www.presidency.ucsb.edu/documents/the-...


---

**Exploring data noise on the end of interview answer**

In [8]:
df_train[["question","interview_answer","url"]]

,question,interview_answer,url
0,How would you respond to the accusation that t...,"Well, look, first of all, the—I am sincere ab...",https://www.presidency.ucsb.edu/documents/the-...
1,Do you think President Xi is being sincere abo...,"Well, look, first of all, the—I am sincere ab...",https://www.presidency.ucsb.edu/documents/the-...
2,Do you believe the country's slowdown and gro...,"Look, I think China has a difficult economic ...",https://www.presidency.ucsb.edu/documents/the-...
3,Are you worried about the meeting between Pre...,"Look, I think China has a difficult economic ...",https://www.presidency.ucsb.edu/documents/the-...
4,Is the President's engagement with Asian coun...,"Well, I hope I get to see Mr. Xi sooner than ...",https://www.presidency.ucsb.edu/documents/the-...
...,...,...,...
3443,Why shouldn't Americans give Democrats a chan...,That's a tricky little question there. [Laugh...,https://www.presidency.ucsb.edu/documents/the-...
3444,Inquiry about the belief regarding the abilit...,"Mike, I believe Iraq will be able to defend, ...",https://www.presidency.ucsb.edu/documents/the-...
3445,Are you resentful that some Republican candid...,"You know, no, I'm not resentful, nor am I res...",https://www.presidency.ucsb.edu/documents/the-...
3446,If you really didn't think that Republicans w...,"You know, no, I'm not resentful, nor am I res...",https://www.presidency.ucsb.edu/documents/the-...


In [10]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Function to get italic sentences, optimized with error handling
def get_italic_sentences(url: str) -> list:
    try:
        response = requests.get(url, timeout=10)  # Added timeout
        response.raise_for_status()  # Raise exception for bad responses
        soup = BeautifulSoup(response.content, 'html.parser')

        # Extract text from the <div> with class "field-docs-content"
        div_content = soup.find('div', class_='field-docs-content')
        if div_content is None:
            return []  # Return an empty list if the div is not found

        exception_list = {"The President.", "Q.", "Inaudible", "inaudible"}

        # Extract unique sentences from <i> or <em> tags, excluding specific phrases
        italic_sentences = {i.get_text(strip=True) for i in div_content.find_all(['i', 'em'])}
        return [sentence for sentence in italic_sentences if sentence not in exception_list]

    except (requests.RequestException, AttributeError) as e:
        print(f"Error retrieving or parsing {url}: {e}")
        return []

# Apply replacement for all rows in a vectorized manner
def clean_interview(row, url_sentences: set):
    unique_sentences = url_sentences.get(row['url'], [])
    interview_answer = row['interview_answer']
    for sentence in unique_sentences:
        interview_answer = interview_answer.replace(sentence, '')
    return interview_answer

# Function to remove unique sentences from the 'interview_answer' column
def remove_noise(df: pd.DataFrame) -> pd.DataFrame:
    url_sentences = {}

    # Create a dictionary to store unique sentences for each URL
    unique_urls = df['url'].unique()

    # Get sentences for each URL (optionally use parallel processing for speedup)
    for url in unique_urls:
        url_sentences[url] = get_italic_sentences(url)

    df['interview_answer'] = df.apply(lambda x: clean_interview(x, url_sentences) , axis=1)

    # Optional: Clean up whitespace after sentence removal
    df['interview_answer'] = df['interview_answer'].str.replace(r'\s+', ' ', regex=True).str.strip()

    return df

In [11]:
# Apply the function to df_train
df_train = remove_noise(df_train)

In [ ]:
df_train

,question,interview_question,interview_answer,label,url
0,How would you respond to the accusation that t...,\nQ. Of the Biden administration. And accused ...,"Well, look, first of all, theI am sincere abou...",Explicit,https://www.presidency.ucsb.edu/documents/the-...
1,Do you think President Xi is being sincere abo...,\nQ. Of the Biden administration. And accused ...,"Well, look, first of all, theI am sincere abou...",General,https://www.presidency.ucsb.edu/documents/the-...
2,Do you believe the country's slowdown and gro...,\nQ. No worries. Do you believe the country's ...,"Look, I think China has a difficult economic p...",Partial/half-answer,https://www.presidency.ucsb.edu/documents/the-...
3,Are you worried about the meeting between Pre...,\nQ. No worries. Do you believe the country's ...,"Look, I think China has a difficult economic p...",Dodging,https://www.presidency.ucsb.edu/documents/the-...
4,Is the President's engagement with Asian coun...,"\nQ. I can imagine. It is evening, I'd like to...","Well, I hope I get to see Mr. Xi sooner than l...",Explicit,https://www.presidency.ucsb.edu/documents/the-...
...,...,...,...,...,...
3443,Why shouldn't Americans give Democrats a chan...,"\nQ. Thank you, Mr. President. With a Republic...",That's a tricky little question there. [] Firs...,Deflection,https://www.presidency.ucsb.edu/documents/the-...
3444,Inquiry about the belief regarding the abilit...,"\nQ. I just wanted to ask you quickly, sir, if...","Mike, I believe Iraq will be able to defend, g...",Explicit,https://www.presidency.ucsb.edu/documents/the-...
3445,Are you resentful that some Republican candid...,\nQ. I understand why you would claim or asser...,"You know, no, I'm not resentful, nor am I rese...",Explicit,https://www.presidency.ucsb.edu/documents/the-...
3446,If you really didn't think that Republicans w...,\nQ. I understand why you would claim or asser...,"You know, no, I'm not resentful, nor am I rese...",Deflection,https://www.presidency.ucsb.edu/documents/the-...


In [12]:
df_train

,question,interview_question,interview_answer,label,url
0,How would you respond to the accusation that t...,\nQ. Of the Biden administration. And accused ...,"Well, look, first of all, theI am sincere abou...",Explicit,https://www.presidency.ucsb.edu/documents/the-...
1,Do you think President Xi is being sincere abo...,\nQ. Of the Biden administration. And accused ...,"Well, look, first of all, theI am sincere abou...",General,https://www.presidency.ucsb.edu/documents/the-...
2,Do you believe the country's slowdown and gro...,\nQ. No worries. Do you believe the country's ...,"Look, I think China has a difficult economic p...",Partial/half-answer,https://www.presidency.ucsb.edu/documents/the-...
3,Are you worried about the meeting between Pre...,\nQ. No worries. Do you believe the country's ...,"Look, I think China has a difficult economic p...",Dodging,https://www.presidency.ucsb.edu/documents/the-...
4,Is the President's engagement with Asian coun...,"\nQ. I can imagine. It is evening, I'd like to...","Well, I hope I get to see Mr. Xi sooner than l...",Explicit,https://www.presidency.ucsb.edu/documents/the-...
...,...,...,...,...,...
3443,Why shouldn't Americans give Democrats a chan...,"\nQ. Thank you, Mr. President. With a Republic...",That's a tricky little question there. [] Firs...,Deflection,https://www.presidency.ucsb.edu/documents/the-...
3444,Inquiry about the belief regarding the abilit...,"\nQ. I just wanted to ask you quickly, sir, if...","Mike, I believe Iraq will be able to defend, g...",Explicit,https://www.presidency.ucsb.edu/documents/the-...
3445,Are you resentful that some Republican candid...,\nQ. I understand why you would claim or asser...,"You know, no, I'm not resentful, nor am I rese...",Explicit,https://www.presidency.ucsb.edu/documents/the-...
3446,If you really didn't think that Republicans w...,\nQ. I understand why you would claim or asser...,"You know, no, I'm not resentful, nor am I rese...",Deflection,https://www.presidency.ucsb.edu/documents/the-...


In [18]:
df_train.to_csv('output.csv', index=False)

In [19]:
# df_train = pd.read_csv('output.csv')
df_train["inaudible"] = df_train['interview_answer'].str.contains('inaudible', case=False)
df_train

,question,interview_question,interview_answer,label,url,inaudible
0,How would you respond to the accusation that t...,\nQ. Of the Biden administration. And accused ...,"Well, look, first of all, theI am sincere abou...",Explicit,https://www.presidency.ucsb.edu/documents/the-...,False
1,Do you think President Xi is being sincere abo...,\nQ. Of the Biden administration. And accused ...,"Well, look, first of all, theI am sincere abou...",General,https://www.presidency.ucsb.edu/documents/the-...,False
2,Do you believe the country's slowdown and gro...,\nQ. No worries. Do you believe the country's ...,"Look, I think China has a difficult economic p...",Partial/half-answer,https://www.presidency.ucsb.edu/documents/the-...,False
3,Are you worried about the meeting between Pre...,\nQ. No worries. Do you believe the country's ...,"Look, I think China has a difficult economic p...",Dodging,https://www.presidency.ucsb.edu/documents/the-...,False
4,Is the President's engagement with Asian coun...,"\nQ. I can imagine. It is evening, I'd like to...","Well, I hope I get to see Mr. Xi sooner than l...",Explicit,https://www.presidency.ucsb.edu/documents/the-...,False
...,...,...,...,...,...,...
3443,Why shouldn't Americans give Democrats a chan...,"\nQ. Thank you, Mr. President. With a Republic...",That's a tricky little question there. [] Firs...,Deflection,https://www.presidency.ucsb.edu/documents/the-...,False
3444,Inquiry about the belief regarding the abilit...,"\nQ. I just wanted to ask you quickly, sir, if...","Mike, I believe Iraq will be able to defend, g...",Explicit,https://www.presidency.ucsb.edu/documents/the-...,False
3445,Are you resentful that some Republican candid...,\nQ. I understand why you would claim or asser...,"You know, no, I'm not resentful, nor am I rese...",Explicit,https://www.presidency.ucsb.edu/documents/the-...,False
3446,If you really didn't think that Republicans w...,\nQ. I understand why you would claim or asser...,"You know, no, I'm not resentful, nor am I rese...",Deflection,https://www.presidency.ucsb.edu/documents/the-...,False


In [21]:
len(df_train[df_train["inaudible"]==True])

45

---

**Exploring unhandled data noise**

1) Affirmative questions

In [ ]:
filtered_no_quest_df = df_train[~df_train['question'].str.contains('\?')]
len(filtered_no_quest_df)

772

In [ ]:
len(df_train[df_train['question'].str.contains('\?')])
len(df_train)

3448

In [ ]:
filtered_no_quest_df

,question,interview_question,interview_answer,label
8,1. Concerns about the lack of communication be...,"\nQ. Well, let me ask you about—you've spent l...",\nThe President. It's not a wedge issue of the...,Explicit
9,2. Inquiry about the reaction of Kyiv regardin...,"\nQ. Well, let me ask you about—you've spent l...",\nThe President. It's not a wedge issue of the...,Explicit
16,ensuring Finland that the U.S. will remain a r...,"\nQ. In Washington, a bipartisan group of Sena...",\nPresident Biden. I absolutely guarantee it. ...,Explicit
18,Concerns about the comments motivating Putin ...,"\nQ. Thank you, Mr. President. You've said tha...","\nPresident Biden. First of all, no one can jo...",Deflection
19,The risk of the war dragging on for years.,"\nQ. Thank you, Mr. President. You've said tha...","\nPresident Biden. First of all, no one can jo...",Dodging
...,...,...,...,...
3403,I wonder what your reaction is to that,\nQ. But the results are being interpreted as ...,"\nThe President. You know, I really haven't—I'...",Explicit
3415,Asking for an explanation of not knowing somet...,\nQ. How could you not know that and not be ou...,"\nThe President. You didn't know it, either.",Dodging
3416,Adjustments to the agenda regarding Social Sec...,"\nQ. Mr. President, you mentioned entitlements...","\nThe President. I told—Ken, I told Hank Pauls...",General
3430,Secretary Rumsfeld Accountability,"\nQ. When you first ran for President, sir, yo...","\nThe President. Peter, you're asking me why I...",Dodging


2) Multiple questions

In [ ]:
df_questionmark_filtered = df_train[df_train['question'].str.count('\?') > 1]
df_questionmark_filtered

,question,interview_question,interview_answer,label
68,"For the 15,000 migrants that Canada will welc...","\nQ. Good afternoon, Mr. President. Good after...","\nPresident Biden. Well, no, I'm not disappoin...",Partial/half-answer
193,What is President Biden's message to the roug...,"\nQ. Thank you, Mr. President. Based on everyt...","\nPresident Biden. Well, I've had discussions—...",Dodging
218,How long should Americans expect to face highe...,\nQ. ——of inflation. Oil prices have been at a...,"\nThe President. Well, look, as you know, Ken,...",Deflection
261,"Why did you tell Jeff [Jeff Zeleny, CNN] that ...",\nQ. Right. We appreciate it. We very much do....,"\nThe President. Well, first of all, the messa...",Partial/half-answer
269,Have you decided who you will nominate to chai...,"\nQ. Well, I'm going to ask a very Bloomberg q...","\nThe President. No, no, and no. No, I'm not g...",Declining to answer
...,...,...,...,...
3274,"Concerning energy matters, 3 days before the c...","\nQ. Good morning. President Calderon, concern...",\nPresident Calderon. The truth of the matter ...,Explicit
3324,\n3. Minority opinion: Is the sentiment of que...,"\nQ. As you know, a growing number of troops a...",\nThe President. I am—what I hear from command...,Dodging
3345,Were the efforts of the Quartet weak or are th...,\nQ. I have a question to the President and th...,\nPresident Bush. I don't know if I'd call thi...,Partial/half-answer
3375,Did you make any representations to the Presid...,"\nQ. Mr. President, the memo from your Nationa...",\nPresident Bush. I will let the Prime Ministe...,Deflection


In [ ]:
len(df_questionmark_filtered)

86